# II. Input -> Extract keyword -> Embed into Vectors


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/23SS_NLP/perfume_embedded_elmo/'

In [3]:
import pandas as pd
import numpy as np

In [4]:
dtype_dict = {
    'C0': 'float'
}

for i in range(0,144):
  globals()[f"perfume_{i}th_vector"] = pd.read_csv(path + 'perfume_{}th_vector.csv'.format(i), dtype=dtype_dict) # dynamic loading

# KeyBERT

In [5]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.8 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=87754163b27771583eff1dd1d4410846e0c8bf75ae4b231fd879d4de9f455cec
  Stored in directory: /root

In [6]:
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')

In [7]:
from tqdm import tqdm
import time

sample

In [8]:
!pip install nlu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.6/532.6 KB 41.0 MB/s eta 0:00:00


In [9]:
!pip install pyspark==3.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 KB 18.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612244 sha256=77b046a34bdf67712e61c5ce31d47bbe5422c1f944744c3136e3a39dd307edd4
  Stored in directory: /root/.cache/pip/wheels/ea/21/84/970b03913d0d6a96ef51c34c878add0de9e4ecbb7c764ea21f
Successfully built pyspark


In [10]:
import nlu

In [11]:
sample_input = "I prefer woody, floral based scent with smoky dry down. However, I would like to avoid any kind of powdery or sharp notes."

In [12]:
# 전처리
import re
def letters_only(x):
  letters_only = re.sub("^[a-zA-Z]","",x)
  letters_only = re.sub(r"[^\w\s]","",letters_only)
  letters_only = letters_only.lower()
  return letters_only


In [13]:
sample_input_after = letters_only(sample_input)

In [14]:
# keyword 추출하는 함수
# 정규표현식 전처리 거친 텍스트 input
def make_keyword(text):
  keywords = kw_extractor.extract_keywords(text, use_mmr = True, top_n=4, stop_words=['scent','fragrance'])
  keyword = pd.DataFrame(keywords, columns=['keyword', 'weight'])
  keyword = list(keyword['keyword'])
  return keyword

In [15]:
sample_keyword = make_keyword(sample_input_after)

In [16]:
print(sample_keyword)

['powdery', 'floral', 'dry', 'woody']


# ELMo

In [17]:
embeddings_sample_input = nlu.load('elmo').predict(sample_input_after,output_level='token')

elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [18]:
embeddings_sample_input.head()

,token,word_embedding_elmo
0,prefer,"[-0.47616660594940186, 1.6042248010635376, 0.0..."
0,woody,"[-0.5818030834197998, -0.41223907470703125, -0..."
0,floral,"[-1.3007525205612183, -0.43067094683647156, 1...."
0,based,"[-0.09254547953605652, 0.673302173614502, 0.82..."
0,scent,"[-0.8762218356132507, -0.3601883351802826, 0.1..."


In [19]:
embeddings_sample_input.reset_index(inplace=True)

In [20]:
embeddings_sample_input['token'][1]

'woody'

In [21]:
sample_keyword

['powdery', 'floral', 'dry', 'woody']

In [22]:
sample_embeddings = {}
for i in range(len(embeddings_sample_input)):
  for j in range(len(sample_keyword)):
    if embeddings_sample_input['token'][i] == sample_keyword[j]:
      sample_embeddings[sample_keyword[j]] = embeddings_sample_input['word_embedding_elmo'][i]
    else:
      j += 1

In [23]:
sample_embedding_vector = pd.DataFrame(sample_embeddings).transpose()
sample_embedding_vector['embeddings'] = ''

for i in range(len(sample_embedding_vector)):
  embedding = []
  for j in range(0,512):
    embedding.append(sample_embedding_vector[j][i])
    sample_embedding_vector['embeddings'][i] = embedding
sample_embedding_vector.drop(range(0,512),axis=1,inplace=True)
sample_embedding_vector.reset_index(inplace=True)
sample_embedding_vector.columns = ['keyword','embeddings']

In [24]:
sample_embedding_vector

,keyword,embeddings
0,woody,"[-0.5818030834197998, -0.41223907470703125, -0..."
1,floral,"[-1.3007525205612183, -0.43067094683647156, 1...."
2,dry,"[0.6724691987037659, 0.010064460337162018, 0.1..."
3,powdery,"[0.2402246743440628, -0.5896206498146057, -0.0..."


In [25]:
sample_embedding_list = [w for w in sample_embedding_vector.iloc[:,1]]

In [26]:
len(sample_embedding_list[0])

512

# III. fit the size of Keyword ELMo to Input-ELMo, in order to calculate cosine similarity

In [27]:
len(sample_embedding_vector)

4

In [28]:
num=len(sample_embedding_vector)

In [29]:
# code for dynamic loading

dtype_dict = {
    'C0': 'float'
}

for i in range(0,144):
  globals()[f"perfume_{i}th_vector"] = pd.read_csv(path + 'perfume_{}th_vector.csv'.format(i), dtype=dtype_dict) # dynamic loading

In [30]:
# for i in range(0,144):
#   globals()[f"perfume_{i}th_vector_truncated"] = globals()[f"perfume_{i}th_vector"].iloc[:num,2]
#   embedding = []
#   for j in range(0,num):
#     embedding.append(globals()[f"perfume_{i}th_vector_truncated"][j])
#     globals()[f"keyword_embedding_to_num{i}"] = embedding

In [31]:
# for i in range(0,144):
#   globals()[f"keyword_embedding_to_num{i}"] = [w for w in globals()[f"perfume_{i}th_vector"].iloc[:num,2]]

In [45]:
for i in range(0,144):
  globals()[f"keyword_embedding_to_num{i}"] = [w for w in globals()[f"perfume_{i}th_vector"].iloc[:num,2]]
  for j in range(0,len(globals()[f"keyword_embedding_to_num{i}"])):
    globals()[f"keyword_embedding_to_num{i}"][j] = globals()[f"keyword_embedding_to_num{i}"][j][1:-1].split(',')
    for k in range(0,len(globals()[f"keyword_embedding_to_num{i}"][j])):
      globals()[f"keyword_embedding_to_num{i}"][j][k] = float(globals()[f"keyword_embedding_to_num{i}"][j][k])

In [46]:
pd.DataFrame(keyword_embedding_to_num0).to_numpy().shape

(4, 512)

In [49]:
len(keyword_embedding_to_num0[0])

512

바꿔줘야함


In [33]:
num

4

In [34]:
for i in range(0,144):
  for j in range(0,num): 
    globals()[f"keyword_embedding_to_num{i}"][j].split(',')

In [35]:
ex = keyword_embedding_to_num0[0].split(',')

In [36]:
ex[1]

' -1.160307765007019'

In [37]:
float(re.sub('[^0-9.-]',"", ex[1]))

-1.160307765007019

In [38]:
[[1,2,3]]

[[1, 2, 3]]

In [39]:
len(keyword_embedding_to_num0)

4

In [40]:
len(pd.DataFrame(keyword_embedding_to_num0).iloc[0,0])

10705

데이터프레임을 만들어서, 첫번째 열은 sample embedding, 두번째 열은 0~143까지의 keyword embedding을 할당해서, 세번째 열에 코사인 유사도를 구한뒤, sorting한다.


In [51]:
import numpy as np
from numpy.linalg import norm
def cosine(A,B):
  cosine = np.dot(A,B)/(norm(A)*norm(B))
  return cosine

In [52]:
def make_df_similarity(x):
  # x is a list. we should conver it into numpy array
  A = pd.DataFrame(x).to_numpy()
  sim_score_each = []
  for i in range(0,144):
    locals()[f"B{i}"] = globals()[f"perfume_{i}th_vector_truncated"].to_numpy()
    locals()[f"sim_score_{i}"] = cosine(A,locals()[f"B{i}"])
    sim_score_each += [locals()[f"sim_score{i}"]]
    sim_score_df = pd.DataFrame(sim_score_each)

In [43]:
# make_df_similarity(sample_embedding_list)

In [54]:
  # A = pd.DataFrame(sample_embedding_list).to_numpy()
  # sim_score_each = []
  # for i in range(0,144):
  #   locals()[f"B{i}"] = pd.DataFrame(globals()[f"keyword_embedding_to_num{i}"]).to_numpy().transpose()
  #   locals()[f"sim_score_{i}"] = cosine(A,locals()[f"B{i}"])
  #   sim_score_each += [locals()[f"sim_score_{i}"]]
  #   sim_score_df = pd.DataFrame(sim_score_each)

ValueError: ignored

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
A = pd.DataFrame(sample_embedding_list).to_numpy()
B0 = pd.DataFrame(keyword_embedding_to_num0).to_numpy()
result0 = cosine_similarity(A,B0)

In [62]:
cosine_similarity(pd.DataFrame([1,2,3,4,5]).to_numpy().transpose(),pd.DataFrame([4,5,2,4,5]).to_numpy().transpose())

array([[0.88695026]])

In [71]:
result0.shape[0]

4

In [72]:
sum([np.sum(array) for array in result0])/(result0.shape[0]*result0.shape[1])

0.19296345447352534

In [73]:
A = pd.DataFrame(sample_embedding_list).to_numpy()
for i in range(0,144):
  globals()[f"B{i}"] = pd.DataFrame(globals()[f"keyword_embedding_to_num{i}"]).to_numpy()

for i in range(0,144):
  globals()[f"result{i}"] = cosine_similarity(A,globals()[f"B{i}"])
  globals()[f"cosine_result{i}"] = sum([np.sum(array) for array in globals()[f"result{i}"]])/(globals()[f"result{i}"].shape[0]*globals()[f"result{i}"].shape[1])

In [86]:
empty_list = []
for i in range(0,144):
  empty_list.append(globals()[f"cosine_result{i}"])
cosine_df = pd.DataFrame(empty_list)
cosine_df.columns = ['cosine_similarity']

In [78]:
data = pd.read_csv('/content/drive/MyDrive/23SS_NLP/perfume_info.csv')


In [81]:
data[['name']]

,name
0,Baccarat Rouge 540
1,Angels' Share
2,Tobacco Vanille
3,By the Fireplace
4,Lost Cherry
...,...
139,Interlude Black Iris
140,Feminité du Bois
141,Halfeti
142,Fève Délicieuse


In [87]:
cosine_df

,cosine_similarity
0,0.192963
1,0.185493
2,0.189322
3,0.191485
4,0.185801
...,...
139,0.190170
140,0.197064
141,0.246262
142,0.261906


In [90]:
cosine_result_final = pd.concat([data[['name']],cosine_df],axis=1)

In [91]:
cosine_result_final

,name,cosine_similarity
0,Baccarat Rouge 540,0.192963
1,Angels' Share,0.185493
2,Tobacco Vanille,0.189322
3,By the Fireplace,0.191485
4,Lost Cherry,0.185801
...,...,...
139,Interlude Black Iris,0.190170
140,Feminité du Bois,0.197064
141,Halfeti,0.246262
142,Fève Délicieuse,0.261906


In [94]:
cosine_result_final.sort_values('cosine_similarity',ascending=False)

,name,cosine_similarity
39,Fucking Fabulous,0.285704
83,Wulong Cha,0.274229
28,Molecule 01,0.273677
25,Oud for Greatness,0.264318
142,Fève Délicieuse,0.261906
...,...,...
58,Kirke,0.143621
132,40 Knots,0.142700
102,African Leather,0.127705
135,Layton Exclusif,0.121967


In [95]:
# result backup
sample_input

'I prefer woody, floral based scent with smoky dry down. However, I would like to avoid any kind of powdery or sharp notes.'

In [96]:
perfume_39th_vector

,Unnamed: 0,keyword,embeddings
0,0,leathery,"[-0.7536596059799194, 0.3149596154689789, -0.3..."
1,1,scorpionbitch,"[-0.8588861227035522, 0.13709276914596558, 0.8..."
2,2,powdery,"[0.24022479355335236, -0.5896204710006714, -0...."
3,3,peppery,"[-0.6082184314727783, 0.15672490000724792, 0.7..."
4,4,vanilla,"[-0.31130504608154297, -0.5989245772361755, 0...."
5,5,sunscreeny,"[-1.0435090065002441, -0.7363113164901733, 0.2..."
6,6,lavender,"[-1.1050264835357666, -0.20609650015830994, 1...."
7,7,cocaine,"[-0.43463748693466187, -0.36749985814094543, 0..."
8,8,sweet,"[-0.2673688530921936, -0.4698314964771271, 0.1..."


In [ ]:
pd.DataFrame(sample_embedding_list).to_numpy().shape

In [ ]:
sample_embedding_list

# Applied to various samples


In [98]:
sample1 = 'Is there something fresh and sharp with cool vibes? Would be nice if it is more like masculine perfume.'
sample2 = 'My favorite scent is sweet vanilla note which is bit dark and forest like.'
sample3 = 'Can you recommend aromatic perfume?'
sample4 = 'I have important meeting with my clients tommorrow. I need a perfume that gives understated, moderate vibes. Something prefered for official occasion, but not too heavy.'
sample5 = 'Is there perfume with honey moon vibes? Sweet and floral fragrance with lovely vibes for date.'

In [100]:
import re
def letters_only(x):
  letters_only = re.sub("^[a-zA-Z]","",x)
  letters_only = re.sub(r"[^\w\s]","",letters_only)
  letters_only = letters_only.lower()
  return letters_only


In [101]:
# keyword 추출하는 함수
# 정규표현식 전처리 거친 텍스트 input
def make_keyword(text):
  keywords = kw_extractor.extract_keywords(text, use_mmr = True, top_n=4, stop_words=['scent','fragrance'])
  keyword = pd.DataFrame(keywords, columns=['keyword', 'weight'])
  keyword = list(keyword['keyword'])
  return keyword

In [102]:
# ELMo Pretrain embedding
for i in range(1,6):
  globals()[f"sample_keyword_{i}"] = make_keyword(letters_only(globals()[f"sample{i}"]))
  globals()[f"embeddings_sample_input_{i}"] = nlu.load('elmo').predict(letters_only(globals()[f"sample{i}"]),output_level = 'token')

elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [103]:
# make sample embedding vectors dynamically
for i in range(1,6):
  globals()[f"embeddings_sample_input_{i}"].reset_index(inplace=True)

for i in range(1,6):
  globals()[f"sample_embeddings_{i}"] = {}
  for k in range(len(globals()[f"embeddings_sample_input_{i}"])):
    for j in range(len(globals()[f"sample_keyword_{i}"])):
      if globals()[f"embeddings_sample_input_{i}"]['token'][k] == globals()[f"sample_keyword_{i}"][j]:
        globals()[f"sample_embeddings_{i}"][globals()[f"sample_keyword_{i}"][j]] = globals()[f"embeddings_sample_input_{i}"]['word_embedding_elmo'][k]
      else:
        j += 1

for i in range(1,6):
  globals()[f"sample_embedding_vector_{i}"] = pd.DataFrame(globals()[f"sample_embeddings_{i}"]).transpose()
  globals()[f"sample_embedding_vector_{i}"]['embeddings'] = ''

for k in range(1,6):
  for i in range(len(globals()[f"sample_embedding_vector_{k}"])):
    embedding = []
    for j in range(0,512):
      embedding.append(globals()[f"sample_embedding_vector_{k}"][j][i])
      globals()[f"sample_embedding_vector_{k}"]['embeddings'][i] = embedding
  globals()[f"sample_embedding_vector_{k}"].drop(range(0,512),axis=1,inplace=True)
  globals()[f"sample_embedding_vector_{k}"].reset_index(inplace=True)
  globals()[f"sample_embedding_vector_{k}"].columns = ['keyword','embeddings']

for i in range(1,6):  
  globals()[f"sample_embedding_list_{i}"] = [w for w in globals()[f"sample_embedding_vector_{i}"].iloc[:,1]]

In [104]:
# code for dynamic loading

dtype_dict = {
    'C0': 'float'
}

for i in range(0,144):
  globals()[f"perfume_{i}th_vector"] = pd.read_csv(path + 'perfume_{}th_vector.csv'.format(i), dtype=dtype_dict) # dynamic loading

In [106]:
# correct the engineering issue of float becoming str. convert it into float again
for i in range(0,144):
  globals()[f"keyword_embedding_to_num{i}"] = [w for w in globals()[f"perfume_{i}th_vector"].iloc[:num,2]]
  for j in range(0,len(globals()[f"keyword_embedding_to_num{i}"])):
    globals()[f"keyword_embedding_to_num{i}"][j] = globals()[f"keyword_embedding_to_num{i}"][j][1:-1].split(',')
    for k in range(0,len(globals()[f"keyword_embedding_to_num{i}"][j])):
      globals()[f"keyword_embedding_to_num{i}"][j][k] = float(globals()[f"keyword_embedding_to_num{i}"][j][k])

In [109]:
# calculating cosine similarity dynamically
for i in range(1,6):
  globals()[f"A{i}"] = pd.DataFrame(globals()[f"sample_embedding_list_{i}"]).to_numpy()

for i in range(0,144):
  globals()[f"B{i}"] = pd.DataFrame(globals()[f"keyword_embedding_to_num{i}"]).to_numpy()

for i in range(0,144):
  for j in range(1,6):
    globals()[f"result{i}_for_{j}th_sample"] = cosine_similarity(globals()[f"A{j}"],globals()[f"B{i}"])
    globals()[f"cosine_result{i}_for_{j}th_sample"] = sum([np.sum(array) for array in globals()[f"result{i}_for_{j}th_sample"]])/(globals()[f"result{i}_for_{j}th_sample"].shape[0]*globals()[f"result{i}_for_{j}th_sample"].shape[1])

In [115]:
# making cosine similarity dataframe dynamically
for i in range(1,6):
  globals()[f"empty_list_{i}"] = []

for j in range(1,6):
  for i in range(0,144):
    globals()[f"empty_list_{j}"].append(globals()[f"cosine_result{i}_for_{j}th_sample"])

for j in range(1,6):
  globals()[f"cosine_df_{j}"] = pd.DataFrame(globals()[f"empty_list_{j}"])
  globals()[f"cosine_df_{j}"].columns = ['cosine_similarity']

In [121]:
for i in range(1,6):
  globals()[f"cosine_result_final_{i}"] = pd.concat([data[['name']], globals()[f"cosine_df_{i}"]],axis=1)
  globals()[f"cosine_result_final_{i}"] = globals()[f"cosine_result_final_{i}"].sort_values('cosine_similarity',ascending=False)

In [128]:
for i in range(1,6):
  print(globals()[f"sample{i}"])

Is there something fresh and sharp with cool vibes? Would be nice if it is more like masculine perfume.
My favorite scent is sweet vanilla note which is bit dark and forest like.
Can you recommend aromatic perfume?
I have important meeting with my clients tommorrow. I need a perfume that gives understated, moderate vibes. Something prefered for official occasion, but not too heavy.
Is there perfume with honey moon vibes? Sweet and floral fragrance with lovely vibes for date.


In [122]:
cosine_result_final_1

,name,cosine_similarity
46,Chocolate Greedy,0.247541
137,La Capitale,0.232257
141,Halfeti,0.222774
47,Erba Pura,0.222262
16,Ani,0.218438
...,...,...
102,African Leather,0.119985
31,Soleil Blanc,0.112844
91,Eau Duelle Eau de Toilette,0.111927
135,Layton Exclusif,0.099157


In [123]:
cosine_result_final_2

,name,cosine_similarity
80,Rehab,0.213158
60,Musk Therapy,0.204241
32,Gypsy Water,0.199823
81,Baraonda,0.197802
5,Layton,0.194733
...,...,...
27,Oud Satin Mood,0.094779
69,Alexandria II,0.092038
120,Fleur de Peau Eau de Parfum,0.090058
31,Soleil Blanc,0.076681


In [124]:
cosine_result_final_3

,name,cosine_similarity
46,Chocolate Greedy,0.168577
141,Halfeti,0.153835
4,Lost Cherry,0.141644
6,Ombré Leather (2018),0.136304
121,Ristretto Intense Café,0.134139
...,...,...
31,Soleil Blanc,0.067761
91,Eau Duelle Eau de Toilette,0.061315
135,Layton Exclusif,0.061292
134,Vanille Fatale,0.056652


In [125]:
cosine_result_final_4

,name,cosine_similarity
46,Chocolate Greedy,0.132706
4,Lost Cherry,0.112604
77,Oajan,0.100682
35,Chergui,0.100559
6,Ombré Leather (2018),0.099944
...,...,...
98,Ebony Wood,0.033232
15,Wood Sage & Sea Salt,0.032161
9,Cedrat Boise,0.025480
28,Molecule 01,0.024605


In [126]:
cosine_result_final_5

,name,cosine_similarity
46,Chocolate Greedy,0.226020
99,"Rosendo Mateu Nº 5 Floral, Amber, Sensual Musk",0.200387
141,Halfeti,0.182189
43,Bois Impérial,0.180319
4,Lost Cherry,0.174703
...,...,...
9,Cedrat Boise,0.085094
133,Armani Prive Rouge Malachite,0.078881
15,Wood Sage & Sea Salt,0.070867
45,Mojave Ghost,0.069826
